In [1]:
"""
3D dataと2D kernelの畳み込みは、fftconvolveでchannel loopすると遅いので、手動でfftしたほうがいい。
paddingなど、細かい点を確認するために、簡単な例を、忘れたら思い出す用に残しておく。
"""

'\n3D dataと2D kernelの畳み込みは、fftconvolveでchannel loopすると遅いので、手動でfftしたほうがいい。\npaddingなど、細かい点を確認するために、簡単な例を、忘れたら思い出す用に残しておく。\n'

### shape

In [2]:
import numpy as np
from scipy.signal import fftconvolve

image = np.array([1,2,3,4,5])
psf   = np.array([1,1,1])

print("full :", fftconvolve(image, psf, mode="full"))
print("same :", fftconvolve(image, psf, mode="same"))
print("valid:", fftconvolve(image, psf, mode="valid"))

full : [ 1.  3.  6.  9. 12.  9.  5.]
same : [ 3.  6.  9. 12.  9.]
valid: [ 6.  9. 12.]


In [ ]:
# つまり、fullの場合の shape = n + k - 1 
# sameの場合の shape = n であり、k - 1 の分だけ両端が切り落とされる。kは奇数なので、各端から、(k-1)/2 ずつ切り落とせばいい。
# 逆に、paddingする場合は、(k-1)/2 ずつ両端に0を追加すればいい。というか、周期的なので、単に前後どちらかに、(k-1)を追加すればいい。

### padding

In [4]:
import numpy as np
from scipy.signal import fftconvolve

# 右端に信号があると、少しだけ漏れ出す
image = np.array([0,0,0,1])
psf   = np.array([1,1,1])

print("fft full:", fftconvolve(image, psf, mode="full"))
print("fft same:", fftconvolve(image, psf, mode="same"))
print("fft valid:", fftconvolve(image, psf, mode="valid"))

fft full: [3.70074342e-17 1.48029737e-16 1.48029737e-16 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
fft same: [1.48029737e-16 1.48029737e-16 1.00000000e+00 1.00000000e+00]
fft valid: [1.48029737e-16 1.00000000e+00]


In [ ]:
# そこでpaddingを追加して、FFTして畳み込むと、改善することを見る
n = len(image)
k = len(psf)

# まず full 畳み込みが「折り返しなし」で収まるFFT長を用意
# full length = n + k - 1
L = n + k - 1

# FFTで full convolution（線形畳み込み）
from numpy.fft import fft, ifft
Y_full = ifft(fft(image, L) * fft(psf, L)).real

# scipy.signal.fftconvolve(image, psf, mode="full") と比較して改善いるか見る
print("manual FFT full :", Y_full)
print("scipy  FFT full  :", fftconvolve(image, psf, mode="full"))

# same は full の中央 n 個を切り出す（scipyと同じ定義）
start = (k - 1) // 2
Y_same = Y_full[start:start + n]
print("\nmanual FFT same :", Y_same)
print("scipy  FFT same  :", fftconvolve(image, psf, mode="same"))

manual FFT full : [0.00000000e+00 7.40148683e-17 7.40148683e-17 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
scipy  FFT full  : [3.70074342e-17 1.48029737e-16 1.48029737e-16 1.00000000e+00
 1.00000000e+00 1.00000000e+00]

manual FFT same : [7.40148683e-17 7.40148683e-17 1.00000000e+00 1.00000000e+00]
scipy  FFT same  : [1.48029737e-16 1.48029737e-16 1.00000000e+00 1.00000000e+00]
